## Make requests with comma separated params

In [ ]:
from unittest import TestCase
import requests
import json
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
def get_request_with_comma_separated_params(base_url, resource, param_name, values):
    params = {param_name: ','.join(map(str, values))}
    response = requests.get(f'{base_url}/{resource}', params=params)
    return response

http://example.com/api/resource?values=1.1%2C2.2%2C3.3


## Test the associated GET request

In [72]:
RUN_FROM = 'bastion' #'uni_wifi'
if RUN_FROM == 'bastion' : URL, HEADERS = 'http://fission:31001/', None
if RUN_FROM == 'uni_wifi': URL, HEADERS =  'http://172.26.135.52:9090/', {'HOST': 'fission'}

In [81]:
def get_model_from_api():
    response = requests.get(URL+'models')
    if response.status_code == 200:
        print('Talked to API, Success')
        return response.json()
    else:
        return {'error': 'Failed to retrieve values from API'}

In [82]:
model = get_model_from_api()
model

{'error': 'Failed to retrieve values from API'}

In [98]:
def get_prediction_from_api(values):
    params = {'predictors': ','.join(map(str, values))}
    response = requests.get(URL, params=params)
    if response.status_code == 200:
        print('Talked to API, Success')
        print(response.text)
        return response.text
    else:
        print(response.text)
        return {'error': 'Failed to retrieve values from API'}

In [99]:
values = [1.1, 2.2]
result = get_prediction_from_api(values)
print(result)

Talked to API, Success




In [ ]:
values = [1.1, 2.2, 3.3]
result = get_prediction_from_api(values)
print(result)

## Tests

In [63]:
URL = 'http://127.0.0.1:5000/api/'

BAD_PARAMS_STR = "{'Status': 400, 'Message': 'Invalid Parameters'}"
ERROR_STR = "{'Status': 500, 'Message': 'Internal Server Error'}"
EMPTY_STR = "{'Status': 200, 'Data': []}"

In [69]:
class APITests(TestCase):

    def __init__(self) -> None:
        super().__init__()

    def test_models_api(self):
        # Missing header
        resp = requests.get(URL+'models').text
        self.assertEqual(resp,BAD_PARAMS_STR)

        # No predictors, returning model coefs
        resp = requests.get(URL+'models/test').json()
        self.assertAlmostEqual(resp['intercept'], 3.)
        self.assertAlmostEqual(resp['coef'][0], 1.)
        self.assertAlmostEqual(resp['coef'][1], 2.)

        # Too many predictors (here 3 instead of 2)
        params = {'predictors': '1.1,2.2,3.3'}
        resp = requests.get(URL+'models/test', params=params).text
        self.assertEqual(resp,BAD_PARAMS_STR)


        # Valid parameters
        params = {'predictors': '1.1,2.2'}
        resp = requests.get(URL+'models/test', params=params).json()
        self.assertAlmostEqual(resp['prediction'])

In [ ]:
test = APITests()
test.test_models_api()

## Generate example model

In [44]:
# Generate some sample data
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.dot(X, np.array([1, 2])) + 3

# Train the linear regression model
model = LinearRegression().fit(X, y)

# Print the coefficients
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

Coefficients: [1. 2.]
Intercept: 3.0000000000000018


In [45]:
with open('lin_reg_model_test.pkl', 'wb') as file:
    pickle.dump(model, file)